In [1]:
import bs4
import lxml

In [3]:
#load the xml file in beutiful soup. make sure you use the lxml-xml parser
soup =bs4.BeautifulSoup(open('./data/bulk-fr/2007/01/FR-2007-01-03.xml'),"lxml-xml")

In [4]:
#This is how we extract rules and proposed rules
rules = soup.FEDREG.RULES.findAll('RULE')
proposedRules = soup.FEDREG.PRORULES.findAll("PRORULE")


In [16]:
rule = rules[0]
rule.FRDOC

<FRDOC> [FR Doc. E6-22436 Filed 12-29-06; 8:45 am] </FRDOC>

In [12]:
def rule_to_dict(rule,_type):
    '''
    Gets a rule, and its type (rule/prorule), return a dictionary with the metadata we need and list of 
    paragraphras
    '''
    preamble = rule.PREAMB
    paragraphs = list(filter(lambda x: len(x)>5,rule.text.split('\n')))
    return {
        'type':_type,
        'agency':preamble.AGENCY.text,
        'title':preamble.SUBJECT.text,
        'doc_id':rule.FRDOC.text , #Use this as an aggregation code
        'paragraphs':paragraphs
    }

In [7]:
#Just to see that it works. If you get an exeption, read out num to see the index of the rule
for num,rule in enumerate(rules):
    
    rule_to_dict(rule,'rule')

NameError: name 'rules' is not defined

In [8]:
#This is fancy python way of going applying the function on all the rules and prorules
ruleResults = map(lambda x: rule_to_dict(x,_type='rule'),rules)
proRuleResults  = map(lambda x: rule_to_dict(x,_type='prorule'),proposedRules)

#Combine them into one list. map is lazy, so we need to call list on each
results = list(ruleResults)+list(proRuleResults)
                  
                  

NameError: name 'rules' is not defined

In [11]:

def rule_results_to_examples(rr):
    '''
    Given a rule, return a list of examples (in lighttag format)
    '''
    paragraphs = rr.pop('paragraphs') #Pop it so we don't append the paragraphs on each example
    results =[] 
    for num,p in enumerate(paragraphs):
        res = {'content':p,'order_code':num} #use doc_id as aggregation code, order_code is the order the ps appeared
        res.update(rr)## add the metadata from the document 
        results.append(res) #append it to the list of results
    return results
        
        

In [100]:
#commented out, but this is what it looks like on one rule
#rule_results_to_examples(results[0])

In [10]:
#Since each rule returns a list, we sum the lists 
allExamples = sum(map(rule_results_to_examples,results),[])

NameError: name 'results' is not defined

In [102]:
len(allExamples)

4769

In [103]:
allExamples[100]

{'content': 'Pursuant to the Congressional Review Act (5 U.S.C. 801 et seq.), the Office of Information and Regulatory Affairs designated this rule as not a major rule, as defined by 5 U.S.C. 804(2).',
 'order_code': 21,
 'type': 'rule',
 'agency': 'DEPARTMENT OF AGRICULTURE',
 'title': 'Rules of Practice and Procedure Governing Formal Rulemaking Proceedings Instituted by the Secretary',
 'doc_id': '[FR Doc. 2019-20585 Filed 9-30-19; 8:45 am]'}

In [12]:
def fr_file_to_example_list(file_path):
    soup =bs4.BeautifulSoup(open(file_path),"lxml-xml")
    rules = soup.FEDREG.RULES.findAll('RULE')
    proposedRules = soup.FEDREG.PRORULES.findAll("PRORULE")
    ruleResults = map(lambda x: rule_to_dict(x,_type='rule'),rules)
    proRuleResults  = map(lambda x: rule_to_dict(x,_type='prorule'),proposedRules)
    results = list(ruleResults)+list(proRuleResults)
    examples = sum(map(rule_results_to_examples,results),[])
    return examples



In [21]:
import os
data =[]
base ='./data/bulk-fr/'
error_paths = []
for root,dirs,files in os.walk(base):
        for f in files:
            
            if f.endswith('xml'):
                path = os.path.join(root,f)
                try:
                    data += fr_file_to_example_list(path)
                except:
                    error_paths.append(path)

In [25]:
import json
json.dump(data,open('fedRegFull.json','w'))

In [107]:
from flashtext import KeywordProcessor

In [115]:
KW = KeywordProcessor(case_sensitive=False)
common = ['convention','treaty','bilateral agreement','international agreement','executive agreement',
       'multilateral agreement','political commitment','soft law','non-binding agreement','non-binding international agreement',
       'memorandum of understanding','foreign policy','international policy','international relations','foreign relations',
       'international negotiations'
      ]

import json
phrases = json.load(open('./phrases.json'))
for w in common:
    KW.add_keyword(w,{'match':w,'match_type':'common'})
for w in phrases:
    KW.add_keyword(w,{'match':w,'match_type':'trigram'})
for w in treaties:
    KW.add_keyword(w['name'],{'match':w['name'],'match_type':'agreement'})
    KW.add_keyword(w['code'],{'match':w['code'],'match_type':'agreement_code'})
    KW.add_keyword(w['country'],{'match':w['country'],'match_type':'agreement_country'})




In [153]:
def has_kws(content):
    return len(KW.extract_keywords(content))>0


def attach_keyword(item):
    kws = KW.extract_keywords(item['content'])
    res =[]
    for kw in kws:
        
        kw.update(item)
        res.append(kw)
    return res

        
    
    
    
# data_with_kws = sum(map(attach_keyword,data),[])

In [117]:
import pandas as pd
D = pd.DataFrame(data_with_kws).drop_duplicates(subset=['content','match'])

In [169]:
gen =filter(lambda x:x,map(attach_keyword,data))


In [170]:
r =[]
for a in gen:
    r+=a




In [174]:
X = pd.DataFrame(r)
len(S.drop_duplicates())

59

In [70]:
S = D.drop_duplicates().set_index(['type','title','content','match'])[['agency','match_type']].sort_index()

In [54]:
S.to_excel('./out1.xlsx')

In [131]:
S

agency  \
type    title                                              content                                            match                                                                                      
prorule Advanced Power Reactor 1400 (APR1400) Design Ce... Part 52 of title 10 of the Code of Federal Regu... 13035                                                      NUCLEAR REGULATORY COMMISSION   
        Agency for International Development Acquisitio... • Home Leave for Qualifying Missions—The propos... Afghanistan                                         AGENCY FOR INTERNATIONAL DEVELOPMENT   
        Agency for International Development Acquisitio... (f) Incentive awards. U.S. Personal Services Co... Foreign Assistance                                  AGENCY FOR INTERNATIONAL DEVELOPMENT   
        Air Plan Approval; Arkansas; Regional Haze Five... In the 2008 Arkansas Regional Haze SIP, the Sta... Georgia                                                  ENVIRONMENTAL PROTECTION AGENCY   
        Air Plan Approval; Illinois; Sulfur Dioxide        In this rule, EPA is finalizing regulatory text... 16-112                                                   ENVIRONMENTAL PROTECTION AGENCY   
        Air Plan Approval; Massachusetts; Infrastructur... \n8 EPA has previously issued findings of failu... 12961                                                    ENVIRONMENTAL PROTECTION AGENCY   
        Air Plan Approval; Oklahoma; Regional Haze Five... \n7 See 79 FR 12944.                               12944                                                    ENVIRONMENTAL PROTECTION AGENCY   
                                                           \n9 See 79 FR 12954.                               12954                                                    ENVIRONMENTAL PROTECTION AGENCY   
                                                           • Does not provide EPA with the discretionary a... 12898                                                    ENVIRONMENTAL PROTECTION AGENCY   
                                                           • Is not an economically significant regulatory... 13045                                                    ENVIRONMENTAL PROTECTION AGENCY   
        Airworthiness Directives; 328 Support Services ... (3) For service information identified in this ... Federal Republic of Germany                                 DEPARTMENT OF TRANSPORTATION   
        Airworthiness Directives; Bombardier, Inc., Air... (3) For service information identified in this ... Canada                                                      DEPARTMENT OF TRANSPORTATION   
                                                           This product has been approved by the aviation ... bilateral agreement                                         DEPARTMENT OF TRANSPORTATION   
        Airworthiness Directives; Diamond Aircraft Indu... Refer to MCAI EASA AD No. 2018-0214, dated Octo... Austria                                                     DEPARTMENT OF TRANSPORTATION   
        Airworthiness Directives; Rolls-Royce Deutschla... This product has been approved by EASA and is a... European Community                                          DEPARTMENT OF TRANSPORTATION   
        Amendments to the Pale Cyst Nematode Regulations   The pale cyst nematode (PCN, Globodera pallida)... Peru                                                           DEPARTMENT OF AGRICULTURE   
        Animal Welfare; Amendments to Licensing Provisi... Under the Animal Welfare Act (AWA or the Act, 7... Guinea                                                         DEPARTMENT OF AGRICULTURE   
        Brucellosis and Bovine Tuberculosis; Update of ... We solicited comments concerning our proposal f... 12832                                                          DEPARTMENT OF AGRICULTURE   
                                                                                                              12833                                     

In [130]:
D[D.match_type=='agreement'][['content','match']].to_excel('tmp.xlsx')